# TensorFlow Data Validation (TFDV): Scaling with Apache Beam

This notebook demonstrates how to use TensorFlow Data Validation (TFDV) with Apache Beam and DataFlow. It reuse the introduction to TFDV made in the previous lab.

- Extract data from BigQuery.
- Compute the summary statistics.
- Explore the computed statistics visually to understand information about the data.
- Infer an initial schema.
- Validate and Update the schema based on a new dataset from BigQuery.
- Save the updated schema to be used as a contract during inference.

### Install dependencies

In [1]:
!pip install tensorflow tensorflow_data_validation google-cloud-bigquery

You should consider upgrading via the '/Users/matthieu/dev/freeldom/mlops-framework/venv/bin/python -m pip install --upgrade pip' command.


### Dataset

This notebook uses [Chicago crime data](https://data.cityofchicago.org/) data published as a public dataset in BigQuery. This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. The data will be extracted with the following columns:

- **date**: Date when the incident occurred. this is sometimes a best estimate.
- **iucr**: The Illinois Unifrom Crime Reporting code.
- **primary_type**: The primary description of the IUCR code.
- **location_description**: Description of the location where the incident occurred.
- **arrest**: Indicates whether an arrest was made.
- **domestic**: Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.
- **district**: Indicates the police district where the incident occurred. 
- **ward**: The ward (City Council district) where the incident occurred.
- **fbi_code**: Indicates the crime classification.
- **year**: Year the incident occurred.



### Imports

In [59]:
import tensorflow as tf 
from google.cloud import bigquery
import tensorflow_data_validation as tfdv
import pandas as pd
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format
import apache_beam as beam 
from datetime import datetime

LOCAL = False # Set the runner type local or GCP
GCS_BUCKET = "freeldom-mlops-deployments" # Set your GCS bucket
PROJECT_ID = 'freeldom' # Set your GCP Project Id
REGION = 'asia-southeast1' # Set the region for Dataflow jobs


CHICAGO_CRIME_TABLE = 'bigquery-public-data.chicago_crime.crime'


## Extract data from BigQuery

Instead of extracting data from 2019 (260,673 records) to generate the schema, let's extract data between 2015 and 2019 (1,331,957 records). We will use the same SQL query to extract the data. However, this time we will use DataFlow to extract the data. We will convert the records into TFRecord format as Tensorflow usually performs better on this type of file, but you could also use CSV if you want to.

In [5]:
def generate_query(year_from: int = None, year_to: int = None, limit: int = None) -> str:
    query = f"""
        SELECT 
            FORMAT_DATE('%Y',  CAST(date AS DATE)) AS crime_year,
            FORMAT_DATE('%b',  CAST(date AS DATE)) AS crime_month,
            FORMAT_DATE('%d',  CAST(date AS DATE)) AS crime_day, 
            FORMAT_DATE('%a',  CAST(date AS DATE)) AS crime_day_of_week, 
            iucr,
            primary_type,
            location_description,
            CAST(domestic AS INT64) AS domestic,
            district,
            ward,
            fbi_code,
            CAST(arrest AS INT64) AS arrest,
        FROM 
          {CHICAGO_CRIME_TABLE}
        """
    if year_from:
        query += f"WHERE year >= {year_from}"
        if year_to:
            query += f" AND year <= {year_to} \n"
    if limit:
        query  += f"LIMIT {limit}"
        
    return query

### Dataflow Pipeline

To convert to TFRecord, we need a function that returns each row of our dataset into a tf.Example record. Then, we define our Apache Beam pipeline.

In [32]:
%%writefile setup.py

from setuptools import setup

setup(
    name='tfdv',
    install_requires=[
      'tensorflow_data_validation'
    ]
)

Writing setup.py


In [52]:
DATA_TYPES = {
    'crime_year': 'STRING',
    'crime_month': 'STRING',
    'crime_day': 'STRING',
    'crime_day_of_week': 'STRING',
    'iucr': 'STRING',
    'primary_type': 'STRING',
    'location_description': 'STRING',
    'fbi_code': 'STRING',
    'domestic': 'INTEGER',
    'district': 'INTEGER',
    'ward': 'INTEGER',
    'arrest': 'INTEGER'
}

def to_example(row, type_mapping):
    features = {}
    for name, value in row.items():
        datatype = type_mapping[name]
        if value is None:
            features[name] = tf.train.Feature()
        elif datatype == 'INTEGER':
            features[name] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        else:
            features[name] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(value)]))
    return tf.train.Example(features=tf.train.Features(feature=features))


In [53]:
def execute_pipeline(query, output_location, runner, type_mapping, args):
    options = beam.options.pipeline_options.GoogleCloudOptions(**args)
    
    with beam.Pipeline(runner, options=options) as pipeline:
        (pipeline 
             | "Extract dataset">> beam.io.Read(beam.io.ReadFromBigQuery(query=query, use_standard_sql=True))
             | "Convert" >> beam.Map(lambda instance: to_example(instance, type_mapping))
             | "Serialize" >> beam.Map(lambda example: example.SerializeToString(deterministic=True))
             | "Save as TFRecords" >> beam.io.WriteToTFRecord(file_path_prefix = output_location, file_name_suffix=".tfrecords")
        )

In [60]:
job_time = datetime.now().strftime('%y%m%d%H%M%S')
root_folder = f"gs://{GCS_BUCKET}/tfdv/{job_name}" if not LOCAL else '.'

In [61]:
runner = "DataflowRunner" if not LOCAL else "DirectRunner"
job_name = f"tfdv-chicago-crime-{job_time}"
year_from =  2015
year_to = 2019
data_folder = root+"/data/"
query = generate_query(year_from, year_to, limit=None if not LOCAL else 10000)
args = {
    'job_name': job_name,
    'runner': runner,
    'project': PROJECT_ID,
    'region': REGION,
    'save_main_session': True,
    'staging_location': root_folder+"/staging/",
    'temp_location': root_folder+"/temp/",
    'setup_file': './setup.py'
}

In [ ]:
execute_pipeline(query, data_folder, runner, DATA_TYPES, args)

## Generating statistics

In [ ]:
job_name = f"tfdv-chicago-crime-stats-{job_time}"
args['job_name'] = job_name
stats_location = root_folder + "/stats/"
options =  beam.options.pipeline_options.GoogleCloudOptions(**args)

_ = tfdv.generate_statistics_from_tfrecord(
    data_location=data_folder, 
    output_path=stats_location,
    stats_options=tfdv.StatsOptions(
        sample_rate=.3
    ),
    pipeline_options = options(**args)
)

## Load statistics

In [ ]:
stats = tfdv.load_statistics(stats_location)
tfdv.visualize_statistics(stats)